In [1]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import IPython.display as ipd

import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))

import librosa
import librosa.display

import os

In [2]:
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    for id, filename in enumerate(filenames):
        print(os.path.join(dirname, filename))
        if id > 10:
            break
            
"""

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    print(dirname)\n    for id, filename in enumerate(filenames):\n        print(os.path.join(dirname, filename))\n        if id > 10:\n            break\n            \n"

In [3]:
i=0
#data, sampling_rate = librosa.load(path_wav+file_wav[i])
#print(data.shape, sampling_rate)

#librosa.display.waveplot(data, sr=sampling_rate)
#ipd.Audio(path_wav+file_wav[i])